In [31]:
import json
from abc import ABC,abstractmethod
from typing import List

In [32]:
DATA_FILE = "hospital_data.json" 
   

In [33]:
class User(ABC):
    def __init__(self,name : str,age : int,contact : str,gender : str):
        self.name = name
        self.age = age
        self.contact = contact
        self.gender = gender

    @abstractmethod
    def get_details(self):
        pass

In [34]:
class Patient(User):
    def __init__(self, name : str, age : int, contact: str, gender:str,disease : str):
        super().__init__(name, age, contact, gender)
        self.disease = disease

    def get_details(self):
        return {
            "Type" : "Patient",
            "Name" : self.name,
            "Age" : self.age,
            "Contact" : self.contact,
            "Gender" : self.gender,
            "Disease" : self.disease
        }

In [35]:
class Doctor(User):
    def __init__(self,name : str,age : int,contact : str,gender : str,specialization : str):
        super().__init__(name,age,contact,gender)
        self.specialization = specialization
    
    def get_details(self):
        return {
            "Type" : "Doctor",
            "Name" : self.name,
            "Age" : self.age,
            "Contact" : self.contact,
            "Gender" : self.gender,
            "Specialization" : self.specialization        
                }

In [36]:
class Nurse(User):
    def __init__(self,name : str,age : int,contact : str,gender : str,department : str):
        super().__init__(name,age,contact,gender)
        self.department = department

    def get_details(self):
        return {
            "Type" : "Nurse",
            "Name" : self.name,
            "Age" : self.age,
            "Gender" : self.gender,
            "Contact" : self.contact,
            "Department" : self.department
        }

In [37]:
def print_user_details(user : User):
    print(json.dumps(user.get_details(),indent=4))
    

In [38]:
class HospitalServices(ABC):
    @abstractmethod
    def book_appointment(self,patient : Patient,doctor : Doctor):
        pass

    @abstractmethod
    def save_data(self):
        pass
    

In [41]:
class Hospital(HospitalServices):
    def __init__(self):
        self.patients : List[Patient] = []
        self.doctors : List[Doctor] = []
        self.appointment = []
        self.load_data()

    def add_patient(self,patient : Patient):
        self.patients.append(patient)
        print(f"Patient {patient.name} Added.")
        self.save_data()

    def add_doctor(self,doctor : Doctor):
        self.doctors.append(doctor)
        print(f"Doctor {doctor.name} added.")
        self.save_data()

    def book_appointment(self, patient : Patient, doctor : Doctor):
        self.appointment.append({"Patient" : patient.name,"Doctor":doctor.name})
        print(f"Appointment booked for {patient.name} with Dr.{doctor.name}")
        self.save_data()

    def save_data(self):
        data = {
            "patients" : [p.get_details() for p in self.patients],
            "doctors" : [d.get_details() for d in self.doctors],
            "appointments" : self.appointment

        }

        with open(DATA_FILE,"w") as f:
            json.dump(data,f,indent=4)

    
    def load_data(self):
        try:
            with open(DATA_FILE,"r") as f:
                data = json.load(f)
                self.patients = [Patient(d["Name"],d["Age"],d["Contact"],d["Gender"],d["Disease"]) for d in data.get("patients",[])]
                self.doctors = [Doctor(d["Name"],d["Age"],d["Contact"],d["Gender"],d["Specialization"]) for d in data.get("doctors")]
                self.appointment = data.get("appointments",[])

        except (FileNotFoundError,json.JSONDecodeError):
            self.patients = []
            self.doctors = []
            self.appointment = []

    def view_data(self):
        print("\n Patients : ")
        for patients in self.patients:
            print(f"{patients.name} (Age : {patients.age},Gender : {patients.gender},Disease : {patients.disease})")

        print("\n Doctors : ")
        for doctor in self.doctors:
            print(f"Dr. {doctor.name} (Specialization : {doctor.specialization})")

        print("\n Appointments : ")
        for appointment in self.appointment:
            print(f"{appointment['Patient']} -> {appointment['Doctor']}")
            
    def delete_data(self):
        category = input("Do you want to delete a (1) Patient or (2) Doctor?\n Enter 1 or 2 : ")

        if category == "1":
            name = input("Enter the nme of the patient to delete : ")
            for patient in self.patients:
                if patient.name.lower() == name.lower():
                    self.patients.remove(patient)
                    print(f"Patient {name} deleted successfully.")
                    self.save_data()
                    return
            print(f"Patient {name} not found.")

        elif category == "2":
            name = input("Enter the name of the doctor to delete : ")
            for doctor in self.doctors:
                if doctor.name.lower() == name.lower():
                    self.doctors.remove(doctor)
                    print(f"Doctor {name} deleted successfully")
                    self.save_data()
                    return 
            print(f"Doctor {name} not found.")

        else:
            print("Invalid choice .. Please enter 1 for Patient or 2 for Doctor.")

            


In [42]:
if __name__ == "__main__":
    hospital = Hospital()

    while True:
        print("\nHOSPITAL MANAGEMENT SYSTEM")
        print("1 Add Patient")
        print("2 Add Doctor")
        print("3 Book Appointment")
        print("4 View Data")
        print("5 Delete Data")
        print("6 Exit")

        choice = input("Enter your choice : ")

        if choice =="1":
            name = input("Enter the Patient name : ")
            age = int(input("Enter the Patient age : "))
            contact = input("Enter the Patient contact : ")
            gender = input("Enter the gender : ")
            disease = input("Enter the Disease : ")
            patients = Patient(name,age,contact,gender,disease)
            hospital.add_patient(patients)

        elif choice == "2":
            name = input("Enter the Doctor name : ")
            age = int(input("Enter the Doctor age : "))
            contact = input("Enter the Doctor contact : ")
            gender = input("Enter the Doctor gender : ")
            specialization = input("Enter the Doctor's Specialization : ")
            doctor = Doctor(name,age,contact,gender,specialization)
            hospital.add_doctor(doctor)

        
        elif choice == "3":
            patient_name = input("Enter the Patient name appointment : ")
            doctor_name = input("Enter the Doctor name for appointment : ")

            patient = next((p for p in hospital.patients if p.name == patient_name),None)
            doctor = next((d for d in hospital.doctors if d.name == doctor_name),None)

            if patient and doctor : 
                hospital.book_appointment(patient,doctor)
            else : 
                print("Invalid Patient and doctor name.Pls Try again.")

        elif choice == "4":
            hospital.view_data()

        elif choice == "5":
            hospital.delete_data()

        elif choice == "6":
            print("Exiting the system.GoodBye!")

            break

        else:
            print("Invalid choice.Try again!")
            
        



HOSPITAL MANAGEMENT SYSTEM
1 Add Patient
2 Add Doctor
3 Book Appointment
4 View Data
5 Delete Data
6 Exit

 Patients : 
Dhruvin (Age : 21,Gender : Male,Disease : Fever)

 Doctors : 
Dr. Jayesh Patel (Specialization : MBBS)

 Appointments : 
Dhruvin -> Jayesh Patel

HOSPITAL MANAGEMENT SYSTEM
1 Add Patient
2 Add Doctor
3 Book Appointment
4 View Data
5 Delete Data
6 Exit
Invalid choice .. Please enter 1 for Patient or 2 for Doctor.

HOSPITAL MANAGEMENT SYSTEM
1 Add Patient
2 Add Doctor
3 Book Appointment
4 View Data
5 Delete Data
6 Exit
Patient Dhruvin deleted successfully.

HOSPITAL MANAGEMENT SYSTEM
1 Add Patient
2 Add Doctor
3 Book Appointment
4 View Data
5 Delete Data
6 Exit

 Patients : 

 Doctors : 
Dr. Jayesh Patel (Specialization : MBBS)

 Appointments : 
Dhruvin -> Jayesh Patel

HOSPITAL MANAGEMENT SYSTEM
1 Add Patient
2 Add Doctor
3 Book Appointment
4 View Data
5 Delete Data
6 Exit

 Patients : 

 Doctors : 
Dr. Jayesh Patel (Specialization : MBBS)

 Appointments : 
Dhruvin ->